In [1]:
import pandas as pd
import xarray as xr
import xoak
import tqdm
from pathlib import Path

In [2]:
from dask.distributed import Client

client = Client("tcp://127.0.0.1:43339")
client

<Client: 'tcp://127.0.0.1:43339' processes=4 threads=16, memory=46.88 GiB>

In [3]:
catalog = pd.read_csv("/home/jovyan/shared_materials/climattr_data_please-dont-copy/nc/catalog.csv")
catalog

,domain_resolution,gcm,rcm,variable,data_file_glob
0,AFR-22,CCCma-CanESM2,CCCma-CanRCM4,pr,/home/jovyan/shared_materials/2025-02_climate_...
1,AFR-22,MOHC-HadGEM2-ES,CLMcom-KIT-CCLM5-0-15,pr,/home/jovyan/shared_materials/2025-02_climate_...
2,AFR-22,MPI-M-MPI-ESM-LR,CLMcom-KIT-CCLM5-0-15,pr,/home/jovyan/shared_materials/2025-02_climate_...
3,AFR-22,MPI-M-MPI-ESM-LR,GERICS-REMO2015,pr,/home/jovyan/shared_materials/2025-02_climate_...
4,AFR-22,NCC-NorESM1-M,CLMcom-KIT-CCLM5-0-15,pr,/home/jovyan/shared_materials/2025-02_climate_...
5,AFR-22,NCC-NorESM1-M,GERICS-REMO2015,pr,/home/jovyan/shared_materials/2025-02_climate_...
6,EUR-11,CCCma-CanESM2,CLMcom-CCLM4-8-17,pr,/home/jovyan/shared_materials/2025-02_climate_...
7,EUR-11,CNRM-CERFACS-CNRM-CM5,CNRM-ALADIN63,pr,/home/jovyan/shared_materials/2025-02_climate_...
8,EUR-11,IPSL-IPSL-CM5A-MR,DMI-HIRHAM5,pr,/home/jovyan/shared_materials/2025-02_climate_...
9,EUR-11,MIROC-MIROC5,UHOH-WRF361H,pr,/home/jovyan/shared_materials/2025-02_climate_...


In [4]:
for nrun in tqdm.tqdm(range(len(catalog))):
    
    ds = xr.open_mfdataset(
        catalog.iloc[nrun].data_file_glob,
        chunks={"time": -1, "rlon": 30, "rlat": 30},
        decode_cf=False,
    )
    if "rlon" not in ds.dims:
        ds = xr.open_mfdataset(
            catalog.iloc[nrun].data_file_glob,
            chunks={"time": -1, "x": 30, "y": 30},
            decode_cf=False,
        )   
    ds = ds.assign_coords(
        lon=ds.lon.isel(time=0, drop=True).compute(),
        lat=ds.lat.isel(time=0, drop=True).compute(),
    )
    ds = ds.chunk({"time": -1})
    ds = ds.assign_coords(
        time=xr.decode_cf(ds[["time"]]).time.compute(),
    )
    ds = ds.drop_vars([v for v in ds.data_vars.keys() if v not in ["pr", "tasmax"]])
    ds = ds.assign_coords(**dict(catalog.iloc[nrun]))
    
    outname = (
        f"{list(ds.data_vars.keys())[0]}"
        f"_{ds.domain_resolution.data[()]}"
        f"_{ds.gcm.data[()]}"
        f"_{ds.rcm.data[()]}"
        ".zarr"
    )

    try:
        ds.drop_encoding().to_zarr(
            Path("/home/jovyan/shared_materials/climattr_data_please-dont-copy/zarr/") / outname,
        )
    except:
        pass
    print(outname, "done")

  4%|▎         | 1/27 [00:05<02:26,  5.64s/it]

pr_AFR-22_CCCma-CanESM2_CCCma-CanRCM4.zarr done


  7%|▋         | 2/27 [00:12<02:34,  6.20s/it]

pr_AFR-22_MOHC-HadGEM2-ES_CLMcom-KIT-CCLM5-0-15.zarr done


 11%|█         | 3/27 [00:13<01:36,  4.02s/it]

...

tasmax_EUR-11_MPI-M-MPI-ESM-LR_DMI-HIRHAM5.zarr done


 96%|█████████▋| 26/27 [1:22:52<05:20, 320.57s/it]

tasmax_EUR-11_MPI-M-MPI-ESM-LR_MPI-CSC-REMO2009.zarr done


100%|██████████| 27/27 [1:26:55<00:00, 193.18s/it]

tasmax_EUR-11_NCC-NorESM1-M_SMHI-RCA4.zarr done
